<h1><b>PROJECT HELPING HANDS Model Training</b></h1>

Members:
1.   Renmar Balana................Leading Developer / Project Manager
2.   Xania Shane Oropesa......Full Stack Developer
3.   Hewey James Lita...........Tester / Front End Developer
4.   Jan Lance Borrero...........Frontend Developer, UI/UX designer
5.   Jiggy Brondial..................Data Gathering / Cleaning

*BSCS 3-A (2022-2023)*

---

## **ROADMAP TO CREATING AN OBJECT DETECTION TFLITE MODEL**


* Collect the dataset of images and label them to get their xml files.

* Install the TensorFlow Object Detection API.

* Generate the TFRecord files required for training. (need generate_tfrecord.py script and csv files for this)

* Edit the model pipeline config file and download the pre-trained model checkpoint.

* Train and evaluate the model.

* Export and convert the model into TFlite(TensorFlow Lite) format.

* Deploy the TFlite model on Android / iOS / IoT devices.




# **0) Checking Prerequisites**

At the time of testing, this notebook works with tensorflow version 2.12, and Python version 3.9.16. 
If the current version of python does not work, try changing to alternative python versions by running the code below.

In [ ]:
!python --version
!sudo update-alternatives --config python3

In [ ]:
!sudo apt install python3-pip

# **1) Import Libraries**